In [ ]:
!pip install flwr
!pip install torch torchvision
!pip install -U flwr[simulation]
!pip install "ray[default]" --ignore-installed

  Using cached ray-2.36.1-cp310-cp310-manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached msgpack-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached protobuf-5.28.2-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached aiohttp-3.10.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import zipfile
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import ndarrays_to_parameters, Context
from flwr.server import ServerApp, ServerConfig
from flwr.server import ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
import flwr as fl
from flwr.client import NumPyClient
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.applications.resnet50 import  ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import VGG16, EfficientNetB6
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import tensorflow as tf
from typing import Dict, List
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from flwr.common import NDArrays, Scalar
import warnings

warnings.filterwarnings("ignore")

In [ ]:

!unzip "BrainTumor.zip"

Archive:  BrainTumor.zip
replace Testing/glioma/Te-glTr_0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_ds="/content/Training"
test_ds="/content/Testing"


In [ ]:
train_giloma="/content/Training/glioma"
train_meningioma="/content/Training/meningioma"
train_pituitary="/content/Training/pituitary"
train_no_tumor="/content/Training/notumor"

test_giloma="/content/Testing/glioma"
test_meningioma="/content/Testing/meningioma"
test_pituitary="/content/Testing/pituitary"
test_no_tumor="/content/Testing/notumor"

train_sets=[train_giloma,train_meningioma,train_pituitary,train_no_tumor]
test_sets=[test_giloma,test_meningioma,test_pituitary,test_no_tumor]


In [ ]:
class_names= os.listdir(train_ds)
n_classes = len(class_names)
print(f"Total Number of Classes : {n_classes} \nClass Names : {class_names}")

In [ ]:
batch_size=32
image_size = (224, 224)

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_ds,target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_ds,target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical')

In [ ]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base of VGG16 to retain pre-trained weights
vgg_base.trainable = False

# Add custom layers on top of the VGG16 base
x = Flatten()(vgg_base.output)  # Flatten the output from the conv base
x = Dense(512, activation='relu')(x)  # Add a fully connected layer
x = Dense(4, activation='softmax')(x)  # 4 output classes

model = Model(inputs=vgg_base.input, outputs=x)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
batch_size=32
image_size = (224, 224)

train_no_tumor_datagen = ImageDataGenerator(rescale=1./255)
test_no_tumor_datagen = ImageDataGenerator(rescale=1./255)

train_no_tumor_generator = train_no_tumor_datagen.flow_from_directory(
    '/content/Training',
    target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical'
)

test_no_tumor_generator = test_no_tumor_datagen.flow_from_directory(
    '/content/Testing',target_size= image_size ,batch_size=batch_size,shuffle=True,class_mode='categorical')

In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])




In [ ]:
history = model.fit(train_no_tumor_generator, epochs= 10, verbose= 1, validation_data=test_no_tumor_generator , shuffle= False)

Epoch 1/10
 47/179 ━━━━━━━━━━━━━━━━━━━━ 44:19 20s/step - accuracy: 0.5249 - loss: 4.8594

In [ ]:
def get_weights(model):
    """Retrieve model weights in a format compatible with Flower."""
    return model.get_weights()

def set_weights(model, weights):
    """Set model weights using Flower parameters."""
    model.set_weights(weights)


In [ ]:
# # Federated Learning Functions
# def set_weights(model, parameters):
#     params_dict = zip(model.state_dict().keys(), parameters)
#     state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
#     model.load_state_dict(state_dict, strict=True)

# def get_weights(model):
#     ndarrays = [val.cpu().numpy() for _, val in model.state_dict().items()]
#     return ndarrays

In [ ]:
# class FlowerClient(NumPyClient):
#     def __init__(self, net, trainset, testset):
#         self.net = net
#         self.trainset = trainset
#         self.testset = testset

#     # Train the model
#     def fit(self, parameters, config):
#         set_weights(self.net, parameters)
#         train_model(self.net, self.trainset)
#         return get_weights(self.net), len(self.trainset), {}

#     # Test the model
#     def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
#         set_weights(self.net, parameters)
#         loss, accuracy = evaluate_model(self.net, self.testset)
#         return loss, len(self.testset), {"accuracy": accuracy}

In [ ]:
# FlowerClient class for Federated Learning
class FlowerClient(NumPyClient):
    def __init__(self, model, trainset, testset):
        self.model = model
        self.trainset = trainset
        self.testset = testset

    # Train the model
    def fit(self, parameters: NDArrays, config: Dict[str, Scalar]):
        set_weights(self.model, parameters)
        self.model.fit(self.trainset, epochs=1)  # Train for one epoch per round
        return get_weights(self.model), len(self.trainset), {}

    # Evaluate the model
    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
        set_weights(self.model, parameters)
        loss, accuracy = self.model.evaluate(self.testset)
        return loss, len(self.testset), {"accuracy": accuracy}

In [ ]:
def evaluate_model(net, testset):
    # Custom evaluation for the four tumor classes
    loss, accuracy = net.evaluate(test_generator)
    _, accuracy_glioma = net.evaluate(test_giloma)
    _, accuracy_meningioma = net.evaluate(test_meningioma)
    _, accuracy_pituitary = net.evaluate(test_pituitary)
    _, accuracy_no_tumor = net.evaluate(test_no_tumor)

    print(f"Test accuracy (all): {accuracy:.4f}")
    print(f"Test accuracy (glioma): {accuracy_glioma:.4f}")
    print(f"Test accuracy (meningioma): {accuracy_meningioma:.4f}")
    print(f"Test accuracy (pituitary): {accuracy_pituitary:.4f}")
    print(f"Test accuracy (no tumor): {accuracy_no_tumor:.4f}")

In [ ]:
# def client_fn(context: Context) -> NumPyClient:
#     # Load the VGG16 model without the top fully connected layers
#     model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#     # Freeze the convolutional base
#     model.trainable = False

#     # Add custom layers on top of the VGG16 base
#     x = Flatten()(model.output)
#     x = Dense(512, activation='relu')(x)
#     x = Dense(4, activation='softmax')(x)

#     # Create the final model
#     model = Model(inputs=model.input, outputs=x)

#     # Compile the model
#     model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

#     # Partition data
#     partition_id = int(context.node_config["partition-id"])
#     client_train = train_sets[int(partition_id)]
#     client_test = testset

#     return FlowerClient(model, client_train, client_test)

In [ ]:
def client_fn(context: Context) -> NumPyClient:
    # Load VGG16 with custom layers
    model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model.trainable = False

    # Add custom layers
    x = Flatten()(model.output)
    x = Dense(512, activation='relu')(x)
    output_layer = Dense(4, activation='softmax')(x)

    # Final model
    model = Model(inputs=model.input, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Partitioning dataset
    partition_id = int(context.node_config["partition-id"])
    client_train = train_generator
    client_test = test_generator

    return FlowerClient(model, client_train, client_test)


In [ ]:
# Server-side configuration
def server_fn(context: Context):
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        initial_parameters=ndarrays_to_parameters(get_weights(model)),
        evaluate_fn=evaluate_model,
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)

# Start the simulation
server = ServerApp(server_fn=server_fn)
run_simulation(server_app=server, client_app=client_fn, num_supernodes=5)

In [ ]:
def server_fn(context: Context):
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.0,
        initial_parameters=ndarrays_to_parameters(initial_parameters),
        evaluate_fn=evaluate,
    )
    config=ServerConfig(num_rounds=3)
    return ServerAppComponents(
        strategy=strategy,
        config=config,
    )

In [ ]:
server = ServerApp(server_fn=server_fn)
run_simulation(server_app=server, client_app=client_fn, num_supernodes=5)